In [ ]:
%pushd ../../

In [ ]:
%env CUDA_VISIBLE_DEVICES=3

In [ ]:
import json

import os
import sys
import tempfile
from tqdm.auto import tqdm

import torch
import torchvision
from torchvision import transforms
from PIL import Image
import numpy as np

torch.cuda.set_device(0)

In [ ]:
from netdissect import setting

In [ ]:
segopts = 'netpqc'

In [ ]:
segmodel, seglabels, _ = setting.load_segmenter(segopts)

In [ ]:
len(seglabels)

In [ ]:
class UnsupervisedImageFolder(torchvision.datasets.ImageFolder):
    def __init__(self, root, transform=None, max_size=None, get_path=False):
        self.temp_dir = tempfile.TemporaryDirectory()
        os.symlink(root, os.path.join(self.temp_dir.name, 'dummy'))
        root = self.temp_dir.name
        super().__init__(root, transform=transform)
        self.get_path = get_path
        self.perm = None
        if max_size is not None:
            actual_size = super().__len__()
            if actual_size > max_size:
                self.perm = torch.randperm(actual_size)[:max_size].clone()
                logging.info(f"{root} has {actual_size} images, downsample to {max_size}")
            else:
                logging.info(f"{root} has {actual_size} images <= max_size={max_size}")

    def _find_classes(self, dir):
        return ['./dummy'], {'./dummy': 0}

    def __getitem__(self, key):
        if self.perm is not None:
            key = self.perm[key].item()
        
        if isinstance(key, str):
            path = key
        else:
            path, target = self.samples[index]
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
            
        if self.get_path:
            return sample, path
        else:
            return sample
            

    def __len__(self):
        if self.perm is not None:
            return self.perm.size(0)
        else:
            return super().__len__()

In [ ]:
class Sampler(torch.utils.data.Sampler):
    def __init__(self, dataset, seg_path):
        self.todos = []
        for path, _ in dataset.samples:
            k = os.path.splitext(os.path.basename(path))[0]
            if not os.path.exists(os.path.join(seg_path, k + '.pth')):
                self.todos.append(path)
                
    def __len__(self):
        return len(self.todos)
    
    def __iter__(self):
        yield from self.todos

In [ ]:
transform = transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ])


In [ ]:
def process(img_path, seg_path, device='cuda', batch_size=128, **kwargs):
    os.makedirs(seg_path, exist_ok=True)

    dataset = UnsupervisedImageFolder(img_path, transform=transform, get_path=True)
    sampler = Sampler(dataset, seg_path)
    loader = torch.utils.data.DataLoader(dataset, num_workers=24, batch_size=batch_size, pin_memory=True, sampler=sampler)  
    
    with torch.no_grad():
        for x, paths in tqdm(loader):
            segs = segmodel.segment_batch(x.to(device), **kwargs).detach().cpu()
            for path, seg in zip(paths, segs):
                k = os.path.splitext(os.path.basename(path))[0]
                torch.save(seg, os.path.join(seg_path, k + '.pth'))
            del segs

In [ ]:
seg_path = 'notebooks/stats/churches/dome2tree/naive'

In [ ]:
import glob

In [ ]:
torch.backends.cudnn.benchmark=True

In [ ]:
!ls notebooks/stats/churches/dome2tree

In [ ]:
process(
    '/data/vision/torralba/ganprojects/placesgan/tracer/baselines/pyflow/dome2tree_all_256/poisson',
    'notebooks/stats/churches/dome2tree_all/poisson',
    batch_size=8,
)

In [ ]:
process(
    '/data/vision/torralba/ganprojects/placesgan/tracer/baselines/pyflow/dome2tree_domeonly_256/naive',
    'notebooks/stats/churches/dome2tree/naive',
    batch_size=8,
)

In [ ]:
process(
    '/data/vision/torralba/ganprojects/placesgan/tracer/baselines/pyflow/dome2tree_domeonly_256/poisson',
    'notebooks/stats/churches/dome2tree/poisson',
    batch_size=8,
)

In [ ]:
process(
    '/data/vision/torralba/ganprojects/placesgan/tracer/baselines/pyflow/dome2tree_domeonly_256/laplace',
    'notebooks/stats/churches/dome2tree/laplace',
    batch_size=8,
)

In [ ]:
process(
    '/data/vision/torralba/distillation/gan_rewriting/results/ablations/stylegan-church-dome2tree-8-1-2001-0.0001-overfit/images',
    'churches/dome2tree_all/overfit',
    batch_size=8)

In [ ]:
process(
    '/data/vision/torralba/ganprojects/placesgan/tracer/utils/samples/domes',
    'churches/domes',
    batch_size=12)

In [ ]:
process(
    '/data/vision/torralba/ganprojects/placesgan/tracer/utils/samples/dome2tree',
    'churches/dome2tree/ours',
    batch_size=8)

In [ ]:
process(
    '/data/vision/torralba/ganprojects/placesgan/tracer/utils/samples/dome2spire',
    'churches/dome2spire/ours',
    batch_size=8)